# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import pandas as pd
from sql_queries import *
from create_tables import *
import psycopg2
from psycopg2 import Error


In [3]:
cur,conn=init_database()
createTables(cur,conn)

successfully creating table


In [4]:
def get_files(path):
    files_path = []
    #traverse all folder and its subfloder from top to bottom
    for root, dirs, files in os.walk(path):
        #creating a pattern so we only get the path that matches that pattern
        json_pattern = os.path.join(root, '*.json')
        file_list = glob.glob(json_pattern)
        for file in file_list:
            files_path.append(os.path.abspath(file))
    return files_path

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
# example file for testing
song_json_files_pathes=get_files("data/song_data")
df = pd.read_json(song_json_files_pathes[0], lines=True)
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [6]:
# extract song_data
df.head()[["song_id","artist_id","title","year","duration"]].values[0]

array(['SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1', "I Didn't Mean To", 0,
       218.93179], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [7]:
try:
    df = pd.read_json(song_json_files_pathes[0], lines=True)
    value=df.head()[["song_id","artist_id","title","year","duration"]].values[0]
    cur.execute(insert_songs_table,value)
except (Exception, Error) as error:
    print("Error while inserting into PostgreSQL", error)    

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [8]:
df[["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]]

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [9]:
try:
    value=df.head()[["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]].values[0]
    cur.execute(insert_artists_table,value)
except (Exception, Error) as error:
    print("Error while inserting into PostgreSQL", error)    

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [10]:
# example file for testing
log_json_files_pathes=get_files("data/log_data")
df1 = pd.read_json(log_json_files_pathes[0], lines=True)
df1.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [11]:
#user_id,first_name,last_name,gender,level
df1[["userId","firstName","lastName","gender","level"]]

,userId,firstName,lastName,gender,level
0,39,Walter,Frye,M,free
1,8,Kaylee,Summers,F,free
2,8,Kaylee,Summers,F,free
3,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free
8,8,Kaylee,Summers,F,free
9,8,Kaylee,Summers,F,free


In [12]:
try:
    value=df1[["userId","firstName","lastName","gender","level"]].values[0]
    cur.execute(insert_users_table,value)
except (Exception, Error) as error:
    print("Error while inserting into PostgreSQL", error)    

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [13]:
#(start_time,hour,day,week,month,year,weekday)
pd.to_datetime(df1.ts, unit='ms')

0    2018-11-01 20:57:10.796
1    2018-11-01 21:01:46.796
2    2018-11-01 21:01:46.796
3    2018-11-01 21:02:12.796
4    2018-11-01 21:05:52.796
5    2018-11-01 21:08:16.796
6    2018-11-01 21:11:13.796
7    2018-11-01 21:17:33.796
8    2018-11-01 21:24:53.796
9    2018-11-01 21:28:54.796
10   2018-11-01 21:42:00.796
11   2018-11-01 21:50:15.796
12   2018-11-01 21:52:05.796
13   2018-11-01 21:55:25.796
14   2018-11-01 22:23:14.796
Name: ts, dtype: datetime64[ns]

In [14]:
start_time=df1.ts
hour=pd.to_datetime(start_time, unit='ms').dt.hour
day=pd.to_datetime(start_time, unit='ms').dt.day
week=pd.to_datetime(start_time, unit='ms').dt.isocalendar().week
month=pd.to_datetime(start_time, unit='ms').dt.month
year=pd.to_datetime(start_time, unit='ms').dt.year
weekday=pd.to_datetime(start_time, unit='ms').dt.weekday
time_data = {'start_time':start_time ,'hour':hour,'hour':hour,'day':day,'week':week,'month':month,'year':year,'weekday':weekday}  
pd.DataFrame(time_data) 


,start_time,hour,day,week,month,year,weekday
0,1541105830796,20,1,44,11,2018,3
1,1541106106796,21,1,44,11,2018,3
2,1541106106796,21,1,44,11,2018,3
3,1541106132796,21,1,44,11,2018,3
4,1541106352796,21,1,44,11,2018,3
5,1541106496796,21,1,44,11,2018,3
6,1541106673796,21,1,44,11,2018,3
7,1541107053796,21,1,44,11,2018,3
8,1541107493796,21,1,44,11,2018,3
9,1541107734796,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [15]:
for index, row in pd.DataFrame(time_data).iterrows():
    try:
        cur.execute(insert_time_table,row)
    except (Exception, Error) as error:
        print("Error while inserting into PostgreSQL", error)    


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [16]:
cur.execute("select * from users")
row =cur.fetchall()
pd.DataFrame(row,columns =["userId","firstName","lastName","gender","level"])

,userId,firstName,lastName,gender,level
0,39,Walter,Frye,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [17]:
for index, row in df1.iterrows():
    try:
        cur.execute(insert_users_table,row[["userId","firstName","lastName","gender","level"]])
    except (Exception, Error) as error:
        print("Error while inserting into PostgreSQL", error)    


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [19]:
for index, row in df1.iterrows():
    try:
        #fetching the matched song_id and artist_id from both song and artist tables
        cur.execute(song_select,(str(row.song),str(row.ts),str(row.artist)))
        result=cur.fetchone()
        song_id,artist_id= None ,None
        if result:
            song_id,artist_id= result
        try:
            cur.execute(insert_songs_plays_table,(row.ts,row.userId,row.level,song_id,artist_id,row.sessionId,row.location,row.userAgent))
        except (Exception, Error) as error:
            print("Error while inserting into songplays table PostgreSQL", error)  

    except (Exception, Error) as error:
        print("Error while selecting song and artist id PostgreSQL", error)  
    

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [85]:
close(cur,conn)

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.